In this project, we will try to generate convincing Shakespearean text by GPT model.

First, let's load the openAi's GPT model, with an additional language model on top (just a linear layer with weights tied to the input embeddings).

In [3]:
import sys
if "google.colab" in sys.modules:
    %pip install -q -U transformers
    %pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.7 MB/s eta 0:00:00


In [4]:
import tensorflow as tf

from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

All model checkpoint layers were used when initializing TFOpenAIGPTLMHeadModel.

All the layers of TFOpenAIGPTLMHeadModel were initialized from the model checkpoint at openai-gpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predictions without further training.


Next we will need a specialized tokenizer for this model. This one will try to use the [spaCy](https://spacy.io/) and [ftfy](https://pypi.org/project/ftfy/) libraries if they are installed, or else it will fall back to BERT's `BasicTokenizer` followed by Byte-Pair Encoding (which should be fine for most use cases).

In [5]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Now let's use the tokenizer to tokenize and encode the prompt text:

In [6]:
tokenizer("hello everyone")

{'input_ids': [3570, 1473], 'attention_mask': [1, 1]}

In [7]:
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(prompt_text,
                                  add_special_tokens=False,
                                  return_tensors="tf")
encoded_prompt

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187]], dtype=int32)>

Easy! Next, let's use the model to generate text after the prompt. We will generate 5 different sentences, each starting with the prompt text, followed by 40 additional tokens. For an explanation of what all the hyperparameters do, make sure to check out this great [blog post](https://huggingface.co/blog/how-to-generate) by Patrick von Platen (from Hugging Face). You can play around with the hyperparameters to try to obtain better results.

In [8]:
num_sequences = 5
length = 40

generated_sequences = model.generate(
    input_ids=encoded_prompt,
    do_sample=True,
    max_length=length + len(encoded_prompt[0]),
    temperature=1.0,
    top_k=0,
    top_p=0.9,
    repetition_penalty=1.0,
    num_return_sequences=num_sequences,
)

generated_sequences

<tf.Tensor: shape=(5, 50), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   544,   557, 12068,   485,   512,   557,   507,   544,
          485,   704,  1074,  7613,   488,   704,  5332,   239,   256,
        40477,   491,  3692,   481,  3016,  1986,   239,   256,   862,
          257,   256,   520,  3488,   239, 40477,   256,   606,   843,
          604,   595,  1925,   599,   485],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   544,   246,  4034,   617,   618,   771, 14803,   488,
         2228, 10250,    14,  5706,   239,   600,   640,  1109,  5995,
          488,   606,  8364,   512,   485,   481,  3453,   498,   481,
         1276,   239,   256, 40477,   246, 11699,   939,   617,  1589,
          481,  4206,  2493,   239, 40477],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   240,   645,   249,   635,   925,   246,  2433,  5934,
          240

Now let's decode the generated sequences and print them:

In [9]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-" * 80)

this royal throne of kings, this sceptred isle is as faithful to you as it is to your own title and your palace.'
 at length the queen spoke.'well?'she demanded. 
'we still have not decided what to
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle is a gift from king derfael and lady rhoslyn. they are both warriors and we invite you to the welcome of the world.'
 a commotion came from outside the castle walls.
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle, if i could make a single request, you would be willing to send me away, and then put me on the line to run the empire. " 
 at the mention of thought, an intense
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle of anoria, should be their humble realm. " 
 " they 'll have him to do with as they see fi

## TODO:

1. Use other GPT versions or Transformer-XL to generate new sentences